In [ ]:
# [설치 후 런타임재시작]
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np


In [4]:
import os
import re
import math
import numpy as np
import pandas as pd
from random import *
from tqdm import tqdm
from itertools import chain
from collections import Counter

import warnings
warnings.filterwarnings(action='ignore')

import seaborn as sns
import matplotlib.pyplot as plt

import nltk
import pdb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pickle

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
df = pd.read_csv('/content/drive/MyDrive/Pal-ette/preprocessed_data/전처리_감성분류.csv')
df

,Unnamed: 0,original,checked,words,emotion
0,0,일은왜해도해도끝이없을까?화가난다그냥내가해결하는게나아남들한테부담주고싶지도않고,일은 왜 해도 해도 끝이 없을까? 화가 난다 그냥 내가 해결하는 게 나아 남들한테 ...,"OrderedDict([('일은', 2), ('왜', 2), ('해도', 2), (...",분노
1,1,이번달에또급여가깎였어!물가는오르는데월급만자꾸깎이니까너무화가나최대한지출을억제해야겠어월...,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나 ...,"OrderedDict([('이번', 4), ('달에', 4), ('또', 4), (...",분노
2,2,회사에신입이들어왔는데말투가거슬려그런애를매일봐야한다고생각하니까스트레스받아잘안맞는사람이...,회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스트...,"OrderedDict([('회사에', 2), ('신입이', 2), ('들어왔는데',...",분노
3,3,직장에서막내라는이유로나에게만온갖심부름을시켜일도많은데정말분하고섭섭해직장사람들과솔직하게...,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은데 정말 분하고 섭섭...,"OrderedDict([('직장에서', 2), ('막내라는', 2), ('이유로',...",분노
4,4,얼마전입사한신입사원이나를무시하는것같아서너무화가나상사인나에게먼저인사하지않아서매일내가먼...,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나 상사 일어나에 게...,"OrderedDict([('얼마', 1), ('전', 1), ('입사한', 1), ...",분노
...,...,...,...,...,...
58266,58266,나랑비슷한시기에결혼하는친구는시댁에서집을해줘서너무부러워우리는대출로시작해우리만뒤처지는것...,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워 우리는 대출로...,"OrderedDict([('나랑', 2), ('비슷한', 2), ('시기에', 2)...",당황
58267,58267,친구한명이결혼해서아이를가졌는데너무행복해보이더라기분이좋지만은않아나도결혼했지만아이도생기...,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라 기분이 좋지만은 않아 ...,"OrderedDict([('친구', 2), ('한', 2), ('명이', 2), (...",당황
58268,58268,남들은결혼전에일억을모았다는데난뭐를한것인지모르겠어자괴감만드네요새집값이너무올라서한숨만나와,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어 자괴감만 드네 요새...,"OrderedDict([('남들은', 2), ('결혼', 2), ('전에', 2),...",당황
58269,58269,나보다결혼을먼저한친구가부러워그친구남편은직장도내남편보다좋고키와외모도저보다훤칠해맞아그친...,나보다 결혼을 먼저 한 친구가 부러워 그 친구 남편은 직장도 내 남편보다 좋고 키와...,"OrderedDict([('나보다', 2), ('결혼을', 2), ('먼저', 2)...",당황


In [8]:
df = df.drop(['Unnamed: 0','original', 'words'], axis=1)
df.head()

,Unnamed: 0,checked,emotion
0,0,일은 왜 해도 해도 끝이 없을까? 화가 난다 그냥 내가 해결하는 게 나아 남들한테 ...,분노
1,1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나 ...,분노
2,2,회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스트...,분노
3,3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은데 정말 분하고 섭섭...,분노
4,4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나 상사 일어나에 게...,분노


In [9]:
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

,checked,emotion
0,일은 왜 해도 해도 끝이 없을까? 화가 난다 그냥 내가 해결하는 게 나아 남들한테 ...,분노
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나 ...,분노
2,회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스트...,분노
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은데 정말 분하고 섭섭...,분노
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나 상사 일어나에 게...,분노


In [10]:
from sklearn.preprocessing import LabelEncoder

y = df["emotion"]

# LabelEncoder 객체 생성
encoder = LabelEncoder()
encoder.fit(y)
labels = encoder.transform(y)

print("labels:", labels)

#인코딩 클래스
print("encoder:", encoder.classes_)
#디코딩원본값
print(encoder.inverse_transform(labels))



labels: [2 2 2 ... 1 1 1]
encoder: ['기쁨' '당황' '분노' '불안' '상처' '슬픔']
['분노' '분노' '분노' ... '당황' '당황' '당황']


In [11]:
df['labels'] = labels
df.tail()

,checked,emotion,labels
58266,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워 우리는 대출로...,당황,1
58267,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라 기분이 좋지만은 않아 ...,당황,1
58268,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어 자괴감만 드네 요새...,당황,1
58269,나보다 결혼을 먼저 한 친구가 부러워 그 친구 남편은 직장도 내 남편보다 좋고 키와...,당황,1
58270,친구들 모두 결혼하고 나만 혼자 남아서 쓸쓸하네 맞아하지만 그렇다고 아무나 하고 결...,당황,1


In [12]:
df.rename(columns={'checked':'text'}, inplace=True)

In [13]:
df['text']

0        일은 왜 해도 해도 끝이 없을까? 화가 난다 그냥 내가 해결하는 게 나아 남들한테 ...
1        이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나 ...
2        회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스트...
3        직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은데 정말 분하고 섭섭...
4        얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나 상사 일어나에 게...
                               ...                        
58266    나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워 우리는 대출로...
58267    친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라 기분이 좋지만은 않아 ...
58268    남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어 자괴감만 드네 요새...
58269    나보다 결혼을 먼저 한 친구가 부러워 그 친구 남편은 직장도 내 남편보다 좋고 키와...
58270    친구들 모두 결혼하고 나만 혼자 남아서 쓸쓸하네 맞아하지만 그렇다고 아무나 하고 결...
Name: text, Length: 58271, dtype: object

In [15]:
df.isnull().sum()

text       0
emotion    0
labels     0
dtype: int64

In [16]:
#중복코드 제거
print(df.duplicated().sum())
duplicated_df = df.duplicated()


516


In [17]:
#불용어 사전
stopwords = """
거야 아 휴 아이구 너무 것 정말 수 그 것 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 하다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 다만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 바깥에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하마터면 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽

"""
# 불용어들을 공백을 기준으로 분리하여 리스트로 변환
stopwords_list = stopwords.split()

# 리스트의 각 단어들을 작은따옴표로 둘러싸서 새로운 리스트 생성
processed_stopwords = ["'{}'".format(word) for word in stopwords_list]

# 작은따옴표로 둘러싸인 불용어 리스트를 다시 공백으로 연결하여 문자열로 변환
processed_stopwords = ' '.join(processed_stopwords)
print(processed_stopwords)

'거야' '아' '휴' '아이구' '너무' '것' '정말' '수' '그' '것' '아이쿠' '아이고' '어' '나' '우리' '저희' '따라' '의해' '을' '를' '에' '의' '가' '으로' '로' '에게' '뿐이다' '의거하여' '근거하여' '입각하여' '기준으로' '예하면' '예를' '들면' '예를' '들자면' '저' '소인' '소생' '저희' '지말고' '하지마' '하지마라' '다른' '물론' '또한' '그리고' '비길수' '없다해서는' '안된다' '뿐만' '아니라' '만이' '아니다' '만은' '아니다' '막론하고' '관계없이' '그치지' '않다' '그러나' '그런데' '하지만' '든간에' '논하지' '않다' '따지지' '않다' '설사' '비록' '더라도' '아니면' '만' '못하다' '하는' '편이' '낫다' '불문하고' '향하여' '향해서' '향하다' '쪽으로' '틈타' '이용하여' '타다' '오르다' '제외하고' '이' '외에' '이' '밖에' '하여야' '비로소' '하다면' '몰라도' '외에도' '이곳' '여기' '부터' '기점으로' '따라서' '할' '생각이다' '하려고하다' '이리하여' '그리하여' '그렇게' '함으로써' '다만' '일때' '할때' '앞에서' '중에서' '보는데서' '으로써' '로써' '까지' '해야한다' '일것이다' '반드시' '할줄알다' '할수있다' '할수있어' '임에' '틀림없다' '한다면' '제' '겨우' '단지' '다만' '할뿐' '딩동' '댕그' '대해서' '대하여' '대하면' '훨씬' '얼마나' '얼마만큼' '얼마큼' '남짓' '여' '얼마간' '약간' '다소' '좀' '조금' '다수' '몇' '얼마' '지만' '하물며' '또한' '그러나' '그렇지만' '하지만' '이외에도' '대해' '말하자면' '뿐이다' '다음에' '반대로' '반대로' '말하자면' '이와' '반대로' '바꾸어서' '말하면' '바꾸어서' '한다면' '만약' '그렇지않으면' '까악' '툭' '딱' '삐걱거리다' '보드득' '비걱거리다' '꽈당

In [18]:
def text_preprocessing(text):
    # 기호 제거
    text = re.sub('[^가-힣]', ' ', text)
    # 불용어 처리
    text = [word for word in text.split() if word not in processed_stopwords]
    text = ' '.join(text)
    return text

In [19]:
df['cleaned_text'] = df['text'].apply(text_preprocessing)
df.head()

,text,emotion,labels,cleaned_text
0,일은 왜 해도 해도 끝이 없을까? 화가 난다 그냥 내가 해결하는 게 나아 남들한테 ...,분노,2,일은 끝이 없을까 화가 난다 그냥 내가 해결하는 나아 남들한테 부담 주고 싶지도
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나 ...,분노,2,이번 달에 급여가 깎였어 물가는 오르는데 월급만 자꾸 깎이니까 화가 최대한 지출을 ...
2,회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스트...,분노,2,회사에 신입이 들어왔는데 말투가 거슬려 애를 매일 봐야 한다고 생각하니까 스트레스받...
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은데 정말 분하고 섭섭...,분노,2,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은데 분하고 섭섭해 직...
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나 상사 일어나에 게...,분노,2,전 입사한 신입사원이 나를 무시하는 같아서 화가 상사 일어나에 먼저 인사하지 않아서...


In [20]:
def create_vocab_and_tokenize(reviews, min_freq=0, max_length=64):
    # 리뷰를 토큰화합니다.
    def tokenize_text(text):
        return text.split()

    tokenized_reviews = [tokenize_text(review) for review in reviews]

    # 각 토큰의 발생 횟수를 세어봅니다.
    token_counts = Counter(chain(*tokenized_reviews))

    # min_freq 이상 출현한 토큰만으로 어휘를 만듭니다.
    vocab = {token: i for i, (token, count) in enumerate(token_counts.items()) if count >= min_freq}

    #어휘에 특수 토큰을 추가합니다
    vocab = {**{'[PAD]': 0, '[UNK]': 1, '[CLS]': 2, '[SEP]': 3}, **vocab}

    # 리뷰를 입력 ID와 attention masks로 변환합니다.
    input_ids = []
    attention_masks = []

    for review in reviews:
        # 리뷰를 토큰화하고 최대 길이에 맞게 잘라냅니다.
        tokens = tokenize_text(review)[:max_length-2]
        input_id = [2] + [vocab.get(token, 1) for token in tokens] + [3]

        attention_mask = [1] * len(input_id) + [0] * (max_length - len(input_id))
        input_id += [0] * (max_length - len(input_id))
        input_ids.append(input_id)
        attention_masks.append(attention_mask)

    return vocab, torch.tensor(input_ids), torch.tensor(attention_masks)

In [21]:
text = df['cleaned_text'].tolist()
emotions = df['labels'].tolist()

In [22]:
vocab, input_ids, attention_masks = create_vocab_and_tokenize(text)

# # a temporary list
# temp_list = y

# # dictionary that maps integer to its string value
# label_dict = {}

# # list to store integer labels
# emotions_int_labels = []

# for i in range(len(temp_list)):
#     label_dict[i] = temp_list[i]
#     emotions_int_labels.append(i)


emotions = torch.tensor(emotions)

input_ids = input_ids.type(torch.LongTensor)
input_ids = input_ids.to(device)
attention_masks = attention_masks.to(device)

emotions = emotions.type(torch.LongTensor)
emotions = emotions.to(device)

# labels = labels.type(torch.LongTensor) # <---- Here (casting)
# labels = labels.to(device)

In [23]:
#데이터셋 생성
dataset = TensorDataset(input_ids, attention_masks, emotions)

# 90%를 train 10%는 validation 사용
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
#random_split 데이터셋 무작위 분할
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [24]:
batch_size = 64
#훈련데이터를 위한 데이터 로더
train_dataloader = DataLoader(
    train_dataset,
    sampler = RandomSampler(train_dataset),  # 무작위 샘플링
    batch_size = batch_size
)
#검증 데이터를 위한 데이터 로더
validation_dataloader = DataLoader(
    val_dataset,
    sampler = SequentialSampler(val_dataset),  # 순차적 샘플링
    batch_size = batch_size
)

In [25]:
maxlen = 512
vocab_size = len(vocab)
max_pred = 20

n_layers = 12
n_heads = 8
d_model = 768
d_ff = 768*4
d_k = d_v = 64

n_segments = 2
lr = 0.0001
epochs = 150

dropout = 0.1

In [26]:
def gelu(x):
  return x *0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

In [27]:
class Embedding(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        self.tok_embedding = nn.Embedding(vocab_size, d_model) # 토큰임베딩 : 주어진 단어나 토큰을 d_model 차원의 벡터로 변환
        self.pos_embedding = nn.Embedding(maxlen, d_model) # 위치 임베딩 :  각 토큰의 위치를  d_model 차원의 벡터로 변환
        self.norm = nn.LayerNorm(d_model) # LayerNorm : 임베딩 벡터를 정규화
        self.dropout = nn.Dropout(dropout) #모델 과적합 방지

    def forward(self, x):
        seq_len = x.size(1) #입력 시퀀스의 길이를 가져옴
        #입력 x의 각 토큰에 대한 위치 정보를 생성
        pos = torch.arange(seq_len,dtype = torch.long, device = x.device)
        pos = pos.unsqueeze(0).repeat(x.size(0), 1)

        #토큰임베딩과 위치임베팅을 합침
        embedding = self.tok_embedding(x) + self.pos_embedding(pos)

        return self.dropout(self.norm(embedding))

In [28]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super().__init__()

        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)

        self.fc = nn.Linear(d_model, d_model)

        self.scale = torch.sqrt(torch.FloatTensor([d_model//n_heads])).to(device)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]

        Q = self.w_q(query)
        K = self.w_k(key)
        V = self.w_v(value)

        Q = Q.view(batch_size, -1, n_heads, d_model//n_heads).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, n_heads, d_model//n_heads).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, n_heads, d_model//n_heads).permute(0, 2, 1, 3)

        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        attention = self.dropout(self.softmax(energy))

        X = torch.matmul(attention, V)
        X = X.permute(0, 2, 1, 3).contiguous()
        X = X.view(batch_size, -1, d_model)

        X = self.fc(X)

        return X


In [29]:
class PositionwiseFeedforward(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv1d(d_model, d_ff, 1)
        self.conv2 = nn.Conv1d(d_ff, d_model, 1)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x => [batch_size, seq_len, d_model]
        x = x.permute(0, 2, 1)
        x = self.dropout(gelu(self.conv1(x)))

        x = self.conv2(x)

        x = x.permute(0, 2, 1)

        return x
      #fc1, fc2대신.

In [30]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder_self_attn = MultiHeadAttention()
        self.encoder_feed_fwd = PositionwiseFeedforward()
        self.layer_norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, input_mask=None):
    # input => [batch_size, seq_len, d_model]

      encoder_outputs = self.layer_norm(input + self.dropout(self.encoder_self_attn(input, input, input, input_mask)))
      encoder_outputs = self.layer_norm(encoder_outputs + self.dropout(self.encoder_feed_fwd(encoder_outputs)))
      return encoder_outputs

In [31]:
class Bert(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        self.embedding = Embedding(vocab_size)
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

        self.linear = nn.Linear(d_model, d_model)
        self.activn = gelu
        self.norm = nn.LayerNorm(d_model)

        # outfeature 개수 조정
        # merged_dataset: '분노', '슬픔', '기쁨', '당황', '불안', '상처'
        self.classifier = nn.Linear(d_model, 8)

    def forward(self, input_ids, attention_mask):
        # 임베딩 레이어를 통과
        embedding_output = self.embedding(input_ids)
        # 각 인코더 레이어를 순차적으로 통과
        for layer in self.layers:
            encoder_output = layer(embedding_output)
        pooled_output = encoder_output[:, 0]
        # 분류를 위해 선형 레이어를 통과
        output = self.classifier(pooled_output)

        return output

In [32]:
model = Bert(vocab_size)
model = model.to(device)
# criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

print(model)

Bert(
  (embedding): Embedding(
    (tok_embedding): Embedding(95317, 768)
    (pos_embedding): Embedding(512, 768)
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (layers): ModuleList(
    (0-11): 12 x EncoderLayer(
      (encoder_self_attn): MultiHeadAttention(
        (w_q): Linear(in_features=768, out_features=768, bias=True)
        (w_k): Linear(in_features=768, out_features=768, bias=True)
        (w_v): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (fc): Linear(in_features=768, out_features=768, bias=True)
        (softmax): Softmax(dim=-1)
      )
      (encoder_feed_fwd): PositionwiseFeedforward(
        (conv1): Conv1d(768, 3072, kernel_size=(1,), stride=(1,))
        (conv2): Conv1d(3072, 768, kernel_size=(1,), stride=(1,))
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_aff

In [33]:
def train_and_validate(model, train_dataloader, validation_dataloader, epochs, optimizer, criterion):

    best_val_accuracy = 0.0
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        print("-" * 10)

        model.train()
        total_loss = 0
        train_preds, train_labels = [], []


        progress_bar = tqdm(train_dataloader, desc="Training", position = 0 , leave=True)
        for step, batch in enumerate(progress_bar):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].long().to(device)

            #그래디언트 초기화
            model.zero_grad()

            outputs = model(b_input_ids, b_input_mask)

            #손실계산
            loss = criterion(outputs.squeeze(), b_labels.long())
            # loss = criterion(outputs.squeeze(), b_labels.float())
            total_loss += loss.item()

            #그래디언트 역전파
            loss.backward()

            #그래디언트 클리핑 - 폭발방지
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            #모델 파라미터 업데이트
            optimizer.step()

            #예측값 레이블 저장
            softmax_probs = torch.nn.functional.softmax(outputs, dim=1)
            train_preds.extend(softmax_probs.squeeze().detach().cpu().numpy().tolist())
            # train_preds.extend(torch.sigmoid(outputs).squeeze().detach().cpu().numpy().tolist())
            train_labels.extend(b_labels.squeeze().detach().cpu().numpy().tolist())

            #진행바 업데이트
            progress_bar.set_postfix({"training_loss": "{:.3f}".format(loss.item() / len(train_dataloader))})

        #평균학습손실
        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Average training loss: {avg_train_loss:.4f}")

        # (바이너리) 학습 정확도 측정
        # train_preds = [1 if pred > 0.5 else 0 for pred in train_preds]
        # train_acc = accuracy_score(train_labels, train_preds)
        # print(f"Train Accuracy: {train_acc:.4f}")


        # train_preds를 텐서로 변환
        # train_preds에서 각 샘플에 대한 최대값을 가지는 클래스 찾기
        predicted_classes = torch.argmax(torch.tensor(train_preds), axis=1)

        # 실제 레이블과 비교하여 정확도 계산
        train_labels_numpy = np.array([label for label in train_labels])
        train_preds_numpy = predicted_classes.numpy()  # 예측된 클래스를 넘파이 배열로 변환

        # 정확도 계산
        train_acc = accuracy_score(train_labels_numpy, train_preds_numpy)
        print(f"Train Accuracy: {train_acc:.4f}")

        train_losses.append(avg_train_loss)
        train_accuracies.append(train_acc)


        #검증모드 설정
        model.eval()
        total_eval_loss = 0
        val_preds, val_labels = [], []

        #검증데이터로 검증 진행
        progress_bar = tqdm(validation_dataloader, desc="Validation", position =0, leave=True)
        for step, batch in enumerate(progress_bar):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].long().to(device)

            #그래디언트 계산 비활성화
            with torch.no_grad():
                outputs = model(b_input_ids, b_input_mask)

                #손실계산
                loss = criterion(outputs.squeeze(), b_labels.long())
                total_eval_loss += loss.item()

                #예측값 및 레이블 저장

                softmax_probs = torch.nn.functional.softmax(outputs, dim=1)

                val_preds.extend(softmax_probs.squeeze().detach().cpu().numpy().tolist())
                val_labels.extend(b_labels.squeeze().detach().cpu().numpy().tolist())

                #진행바 업데이트
                progress_bar.set_postfix({"validation_loss": "{:.3f}".format(loss.item() / len(batch))})

        #평균검증손실

        avg_val_loss = total_eval_loss / len(validation_dataloader)

        # train_preds에서 각 샘플에 대한 최대값을 가지는 클래스 찾기
        val_predicted_classes = torch.argmax(torch.tensor(val_preds), axis=1)

        # 실제 레이블과 비교하여 정확도 계산
        val_labels_numpy = np.array([label for label in val_labels])
        val_preds_numpy = val_predicted_classes.numpy()  # 예측된 클래스를 넘파이 배열로 변환

        #검증 정확도 측정 val_preds = [1 if pred > 0.5 else 0 for pred in val_preds]
        val_acc = accuracy_score(val_labels_numpy, val_preds_numpy)
        print(f"Validation Accuracy: {val_acc:.4f}")

        val_losses.append(avg_val_loss)
        val_accuracies.append(val_acc)


        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            # Save the model if the current validation accuracy is the best
            if not os.path.exists("/content/datamerged_model.pth"):
              print("File does not exist.")
              with open("/content/datamerged_model.pth", "w") as f:
                pass
            torch.save(model.state_dict(),  "/content/datamerged_model.pth")

            print(f"Model saved: Improved validation accuracy to {val_acc:.4f}")

    return train_losses, val_losses, train_accuracies, val_accuracies


In [37]:
import tensorflow as tf


check_point_path = '/content/drive/MyDrive/1조_감성대화/BERT_checkpoint'
mc = tf.keras.callbacks.ModelCheckpoint(check_point_path,
                                        monitor='val_loss',
                                        save_best_only=True)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)


In [ ]:
train_losses, val_losses, train_accuracies, val_accuracies = train_and_validate(model, train_dataloader, validation_dataloader, epochs, optimizer, criterion)

Epoch 1/150
----------


Training: 100%|██████████| 820/820 [01:24<00:00,  9.67it/s, training_loss=0.001]


Average training loss: 1.3378
Train Accuracy: 0.4816


Validation: 100%|██████████| 92/92 [00:06<00:00, 13.33it/s, validation_loss=0.432]


Validation Accuracy: 0.5012
Model saved: Improved validation accuracy to 0.5012
Epoch 2/150
----------


...

Training: 100%|██████████| 820/820 [01:25<00:00,  9.65it/s, training_loss=0.000]


Average training loss: 0.0844
Train Accuracy: 0.9716


Validation: 100%|██████████| 92/92 [00:06<00:00, 13.34it/s, validation_loss=0.970]


Validation Accuracy: 0.5599
Epoch 95/150
----------


Training:  64%|██████▍   | 527/820 [00:54<00:30,  9.70it/s, training_loss=0.000]

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Loss over epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Accuracy over epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
PATH = "/content/drive/MyDrive/Pal-ette/datamerged_model_1.pth"
PATH_empathy = "/content/drive/MyDrive/Pal-ette/trained_models/empathy_model.pth"